In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def read_df(filename):
    df = pd.read_json(filename, lines = True)
    df["reviewTime"] = pd.to_datetime(df["reviewTime"])
    df = df.drop_duplicates(subset = ["reviewTime", "reviewerID", "asin"])
    df = df.reset_index(drop = True)
    df = df.groupby("asin").filter(lambda x: len(x) >= 5)
    df = df.reset_index(drop = True)
    return df

In [5]:
df = read_df("Video_Games.json")
#df.groupby("asin").filter(lambda x: len(x) >= 500).groupby("asin")["overall"].mean().nsmallest(5)
df.overall.value_counts()

5    286563
4     88365
3     46477
1     29093
2     22781
Name: overall, dtype: int64

In [ ]:
df2 = df.iloc[:10000,:]

df2["polarity"] = np.where(df2.overall <= 3, 0, 1)

df2["!"] = df2["reviewText"].copy()
for i in range(len(df2["reviewText"])):
    if "!" in df2.loc[i, "reviewText"]:
        df2.loc[i, "!"] = True
    else: 
        df2.loc[i, "!"] = False
        
sm.Logit(endog = df2["polarity"], exog = sm.add_constant(df2["!"]).astype(int)).fit().summary()